# Eat Safe, Love

## Notebook Set Up

In [13]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [12]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {count}")

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)



Number of establishments with a hygiene score of 20: 41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': {'$toInt': '$RatingValue'},
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('654b015e0b0888a157f4d934'),
 'geocode': {'latitude': '50.769705', 'longitude': '0.27694'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/11068

In [14]:
# Convert the result to a Pandas DataFrame
query_result = establishments.find(query)
df = pd.DataFrame(list(query_result))

# Display the number of rows in the DataFrame
num_rows = len(df)
print(f"Number of rows in the DataFrame: {num_rows}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 41
First 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  654b015e0b0888a157f4d934   110681                  0   
1  654b015e0b0888a157f4dcb7   612039                  0   
2  654b015e0b0888a157f4dfbe   730933                  0   
3  654b015e0b0888a157f4e1ae   172735                  0   
4  654b015e0b0888a157f4e1b8   172953                  0   
5  654b015e0b0888a157f4eb58   512854                  0   
6  654b015e0b0888a157f4ed78  1537089                  0   
7  654b015e0b0888a157f502a8   155648                  0   
8  654b015e0b0888a157f506e8  1012883                  0   
9  654b015f0b0888a157f50efe   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza 

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [20]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "LocalAuthorityName": "London",
    "RatingValue": {"$gte": 4}
}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Result: {count}")

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

Result: 0
None


In [21]:
# Convert the result to a Pandas DataFrame
query_result = establishments.find(query)
df = pd.DataFrame(list(query_result))

# Display the number of rows in the DataFrame
num_rows = len(df)
print(f"Number of rows in the DataFrame: {num_rows}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [29]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.152225
longitude = 1.387974

query = {
    "RatingValue": 5
}

sort = [("scores.Hygiene", -1)]

results = establishments.find(query).sort(sort)

# Print the results
for result in results:
    pprint(result)

In [30]:
# Convert result to Pandas DataFrame
df_top5 = pd.DataFrame(list(results))

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [33]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

pipeline = [
    {
        "$match": {
            "scores.Hygiene": 0  # Match establishments with a hygiene score of 0
        }
    },
    {
        "$group": {
            "_id": "$LocalAuthorityName",  # Group by Local Authority
            "count": {"$sum": 1}  # Count the number of matches in each group
        }
    },
    {
        "$sort": {
            "count": -1  # Sort from highest to lowest count
        }
    }
]

# Execute the aggregation pipeline
#result = establishments.aggregate(pipeline)
result_list = list(result)

# Print the number of documents in the result
count = len(result_list)
print(f"Number of documents in the result: {count}")


# Print the first 10 results
for i, doc in enumerate(result):
    if i < 10:
        print(doc)


Number of documents in the result: 55


In [34]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result_list)

# Display the number of rows in the DataFrame
num_rows = len(df)
print(f"Number of rows in the DataFrame: {num_rows}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))


Number of rows in the DataFrame: 55
First 10 rows of the DataFrame:
               _id  count
0           Thanet   1130
1        Greenwich    882
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542
